In [1]:
import pandas as pd

df = pd.read_csv('senate_data.csv', parse_dates=['Transaction Date'])
#only working on Stocks
df = df[df['Type'] == 'Stock']

In [2]:
#Drop exchanges
df = df[df['Buy/Sell'] != 'Exchange']

In [3]:
#Just going to drop the irregular rows
df = df[df['Stock'].apply(lambda x: len(x) < 5)]

In [4]:
#fill NaN in type to stock
df['Type'].fillna('Stock', inplace=True)

In [ ]:
#get stock data for each day, drop non found (also only US)
import datetime
import yfinance as yf

#replace FB with META
df['Stock'] = df['Stock'].replace('FB', 'META')

for i, row in df.iterrows():
    stock = row['Stock']
    transaction_date = row['Transaction Date'].date()
    ticker = yf.Ticker(stock)
    data = ticker.history(start=transaction_date, end=transaction_date+datetime.timedelta(days=1))
    if data.empty:
        df.drop(i, inplace=True) 
    else:
        df.loc[i, 'Price'] = data['Close'].values[0]

#resave as cleaned csf and save prices as json to be used for later analysis
df.to_csv('senate_data_clean.csv', index=False) 

#irregular stocks to fix
irregular = df[df['Stock'].apply(lambda x: len(x) > 5)]
#drop all rows that are weird in the stock column;
irregular = irregular[~irregular['Stock'].str.contains('bond|bds|due|%', case=False)]

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import json
#try to search up ticker
driver = webdriver.Firefox()
names = {}
#workaround for startup web scraping block - going slowly
count = 0
sleep = 0
for stock in irregular['Stock']:
    count += 1
    sleep += 1
    driver.get('https://www.startpage.com')
    search_bar = driver.find_element(By.ID, 'q') 
    og = stock
    if '\n' in stock:
        stock = stock.split('\n')[0]
    search_bar.send_keys(f"{stock} stock symbol yahoo finance")
    search_bar.send_keys(Keys.RETURN)
    time.sleep(6) 
    try: #yahoo
        search_results = driver.find_element(By.XPATH, "//a[contains(@href, 'https://finance.yahoo.com/quote/')]")
        search_results = search_results.text.split('/')[-2]
        names[og] = search_results
    except:
        names[og] = 'N/A'
    time.sleep(6)
    if count == 20:
        count = 0
        time.sleep(600)
    if sleep == 200:
        sleep = 0
        time.sleep(6000)
driver.quit()
with open('clean_irregulars.json', 'w') as f:
    json.dump(names, f)